In [208]:
import jax

In [200]:
import gymnax
from tqdm.auto import tqdm
env, params = gymnax.make('Freeway-MinAtar')

In [201]:
import jax
from jax.random import split

rng = jax.random.PRNGKey(0)
rng, _rng = split(rng)

obs, state = jax.vmap(env.reset, (0, None))(split(rng, 1024), params)
a = []
for i in tqdm(range(1000)):
    rng, _rng = split(rng)
    act = jax.random.randint(_rng, (1024, ), 0, env.action_space(params).n)
    rng, _rng = split(rng)
    obs, state, rew, done, info = jax.vmap(env.step, (0, 0, 0, None))(split(_rng, 1024), state, act, params)
    a.append(rew)

a = jnp.stack(a).flatten()



  0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
plt.hist(np.asarray(a));
plt.yscale('log')

In [ ]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import pickle
import json



In [ ]:
import matplotlib.colors as mcolors
mcolors.TABLEAU_COLORS


In [ ]:
def viz_training(dir_exp):
    with open(f'{dir_exp}/rets.pkl', 'rb') as f:
        rets = pickle.load(f)
        rets = np.asarray(rets)
    plt.plot(rets[0])


viz_training('../data/exp_main/train/name=csmdp;d_state=8;d_obs=8;n_acts=4;delta=F;trans=linear;rew=linear;mrl=4x64')
viz_training('../data/exp_main/train/name=csmdp;d_state=8;d_obs=8;n_acts=4;delta=F;trans=linear;rew=goal;mrl=4x64')
viz_training('../data/exp_main/train/name=csmdp;d_state=8;d_obs=8;n_acts=4;delta=F;trans=mlp;rew=linear;mrl=4x64')

plt.ylabel('Return')
plt.xlabel('Training Iteration')



In [ ]:
def viz_dir(dir_exp, bar_chart=True):
    # for d in glob.glob(f'{dir_exp}/train/*'):
    #     print(d)
    #     with open(f'{d}/config.json', 'r') as f:
    #         cfg = json.load(f)
    #         print(cfg)
    
    plt.figure(figsize=(12, 8))
    i_plt = 0
    for test_dir in sorted(os.listdir(f'{dir_exp}/test/')):
        i_plt += 1
        plt.subplot(2, 3, i_plt)
        for i, train_dir in enumerate(sorted(os.listdir(f'{dir_exp}/test/{test_dir}/'))):
            with open(f'{dir_exp}/test/{test_dir}/{train_dir}/rets.pkl', 'rb') as f:
                rets = pickle.load(f)
                rets = np.asarray(rets)
                if bar_chart:
                    plt.bar(i, rets[0, -100:].mean(), label=train_dir)
                else:
                    plt.plot(rets[0], label=train_dir)
        plt.title(test_dir)
        
        # print(test_dir)
        # print(rets)
    plt.gcf().supxlabel('Algorithm' if bar_chart else 'Training Iteration')
    plt.gcf().supylabel('End Return' if bar_chart else 'Return')
    plt.tight_layout()
    handles, labels = plt.gca().get_legend_handles_labels()
    plt.figlegend(handles, labels, bbox_to_anchor=(1, 0))
    


viz_dir('../data/exp_main/', bar_chart=True)


In [ ]:
from dataclasses import dataclass

@dataclass
class Dog():
    legs: int
    
@dataclass
class Cat():
    MyDog: object
    def doit(self):
        self.my_dog = self.MyDog()
        print(self.MyDog)
        print(self.my_dog)

from functools import partial
cat = Cat(partial(Dog, legs=4))
cat.doit()
    
    

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from ppo_metagrid import make_train

In [ ]:
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt

In [ ]:
if __name__ == "__main__":
    config = {
        "LR": 2.5e-4,
        "NUM_ENVS": 16*4,
        "NUM_STEPS": 128,
        "TOTAL_TIMESTEPS": 16*5e5,
        "UPDATE_EPOCHS": 4,
        "NUM_MINIBATCHES": 4,
        "GAMMA": 0.99,
        "GAE_LAMBDA": 0.95,
        "CLIP_EPS": 0.2,
        "ENT_COEF": 0.01,
        "VF_COEF": 0.5,
        "MAX_GRAD_NORM": 0.5,
        "ACTIVATION": "tanh",
        "ENV_NAME": "CartPole-v1",
        "ANNEAL_LR": False,
        "DEBUG": True,
    }
    rng = jax.random.PRNGKey(30)
    train_fn = jax.jit(jax.vmap(make_train(config)))
    rng, *_rng = jax.random.split(rng, 1 + 32)
    out = train_fn(jnp.arange(len(_rng)), jnp.stack(_rng))
    metrics = out["metrics"]
    print(jax.tree_map(lambda x: x.shape, metrics))
    # rets = metrics["returned_episode_returns"]  # n_seed, n_iters, n_steps, n_envs
    # import matplotlib.pyplot as plt
    # 
    # n_iters = rets.shape[1]
    # steps = jnp.arange(n_iters) * config["NUM_STEPS"] * config["NUM_ENVS"]
    # plt.plot(steps, jnp.mean(rets, axis=(0, 2, 3)), label='mean')
    # plt.plot(steps, jnp.median(rets, axis=(0, 2, 3)), label='median')
    # plt.plot(steps, jnp.mean(rets, axis=(2, 3)).T, c='gray', alpha=0.1)
    # plt.legend()
    # plt.ylabel('Return')
    # plt.xlabel('Env Steps')
    # plt.show()
    # 
    # plt.plot(jnp.mean(rets[:, :10, :, :], axis=(0, 1, 3)), label='start of training')
    # plt.plot(jnp.mean(rets[:, -10:, :, :], axis=(0, 1, 3)), label='end of training')
    # plt.legend()
    # plt.xlabel('in context steps')
    # plt.show()
    

In [ ]:
rews = metrics  # seed, iter, step, env
rets = rews.sum(axis=2)  # seed, iter, env
rews.shape, rets.shape

In [ ]:
n_iters = rews.shape[1]
iters = jnp.arange(n_iters) * config["NUM_STEPS"] * config["NUM_ENVS"]

plt.plot(iters, jnp.mean(rets.mean(axis=-1), axis=0), label='mean')
plt.plot(iters, jnp.median(rets.mean(axis=-1), axis=0), label='median')
plt.plot(iters, rets.mean(axis=-1).T, c='gray', alpha=0.1)
plt.legend()
plt.ylabel('Return')
plt.xlabel('Env Steps')
plt.title('MetaGridEnv 64x128, Agent: MetaRLLinearTransformer')
plt.show()


In [ ]:
plt.plot(jnp.mean(rews[:, :10].mean(axis=(1, 3)), axis=0), c=[.5, 0, 0, 1], label='mean start of training')
plt.plot(jnp.mean(rews[:, -10:].mean(axis=(1, 3)), axis=0), c=[0, .5, 0, 1], label='mean end of training')

plt.plot(rews[:, :10].mean(axis=(1, 3)).T, c=[.5, 0, 0, .1])
plt.plot(rews[:, -10:].mean(axis=(1, 3)).T, c=[0, .5, 0, .1])

plt.legend()
plt.ylabel('Per-Step Reward')
plt.xlabel('In-Context Steps')
plt.title('MetaGridEnv 64x128, Agent: MetaRLLinearTransformer')
plt.show()
